<a href="https://colab.research.google.com/github/jporeilly/Workshop--Data-Integration/blob/main/Copy_of_credit_card_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit Card Fraud Detection - H2O AutoML (Fast Version)

**Optimized for speed - trains only 5 models**

- ⚡ Training time: ~10-15 minutes
- 🎯 Top 5 best algorithms only
- ✅ All essential features included
- 🚀 Python 3.12 compatible

In [ ]:
# Install H2O
!pip install h2o -q
print("✅ Installation complete!")

In [ ]:
# Import libraries
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize H2O
h2o.init(max_mem_size='4G', nthreads=-1)
print("\n✅ H2O initialized!")

In [ ]:
# Upload data
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load data
data = pd.read_csv('TPOT.csv', sep=';', header=None)

data.columns = [
    'first_time_customer', 'order_dollar_amount', 'num_items', 'age',
    'web_order', 'total_transactions', 'hour_of_day',
    'billing_shipping_match', 'fraud'
]

# Convert target to string
data['fraud'] = data['fraud'].astype(str)

print(f"Dataset: {data.shape[0]:,} rows × {data.shape[1]} columns")
print(f"Fraud cases: {(data['fraud']=='1').sum():,} ({(data['fraud']=='1').mean()*100:.2f}%)")
data.head()

In [ ]:
# Quick class distribution check
fraud_dist = data['fraud'].value_counts()

plt.figure(figsize=(10, 4))
fraud_dist.plot(kind='bar', color=['green', 'red'])
plt.title('Class Distribution', fontweight='bold')
plt.xlabel('Class (0=Normal, 1=Fraud)')
plt.ylabel('Count')
plt.xticks([0, 1], ['Normal', 'Fraud'], rotation=0)
for i, v in enumerate(fraud_dist):
    plt.text(i, v + 50, f'{v:,}', ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

## Convert to H2O and Split Data

In [ ]:
# Convert to H2O frame
hf = h2o.H2OFrame(data)
hf['fraud'] = hf['fraud'].asfactor()

# Split: 75% train, 25% test
train, test = hf.split_frame(ratios=[0.75], seed=42)

print(f"Training: {train.nrows:,} rows")
print(f"Test:     {test.nrows:,} rows")

# Define features and target
x = train.columns
x.remove('fraud')
y = 'fraud'

## Run Fast H2O AutoML (Top 5 Models Only)

**Configuration:**
- Maximum 5 models
- 15 minutes max runtime
- Includes: GBM, XGBoost, GLM, DRF (Random Forest), DeepLearning

In [ ]:
# Fast configuration
start_time = datetime.now()

print("\n🚀 FAST H2O AutoML")
print("="*70)
print("Configuration:")
print("  Max models: 5 (for speed)")
print("  Max runtime: 15 minutes")
print("  Algorithms: GBM, XGBoost, GLM, DRF, DeepLearning")
print("  Balance classes: True")
print("\nTraining...\n")

aml = H2OAutoML(
    max_models=5,                   # ⚡ Only 5 models for speed!
    max_runtime_secs=900,           # 15 minutes max
    seed=42,
    balance_classes=True,           # Handle imbalanced data
    nfolds=5,                       # 5-fold CV
    sort_metric='AUC',

    # Best algorithms for fraud detection
    include_algos=[
        'GBM',           # Gradient Boosting (usually best)
        'XGBoost',       # Extreme Gradient Boosting
        'GLM',           # Generalized Linear Model (fast)
        'DRF',           # Distributed Random Forest (tree-based)
        'DeepLearning'   # Neural Network
    ],

    verbosity='info'
)

# Train
aml.train(x=x, y=y, training_frame=train)

duration = (datetime.now() - start_time).total_seconds()
print(f"\n✅ Training completed in {duration/60:.1f} minutes!")
print("="*70)

## Leaderboard

In [ ]:
# View leaderboard
lb = aml.leaderboard
print("\n📊 ALL MODELS RANKED BY PERFORMANCE")
print("="*70)
print(lb)
print("="*70)
print(f"\nTotal models trained: {lb.nrows}")
print(f"Best model: {lb.as_data_frame()['model_id'][0]}")

## Best Model Performance

In [ ]:
# Get best model
best = aml.leader

print(f"\n🏆 BEST MODEL: {best.model_id}")
print(f"Algorithm: {best.algo}")

# Performance on test set
perf = best.model_performance(test)

print("\n📈 TEST SET PERFORMANCE")
print("="*70)
print(f"AUC:       {perf.auc():.4f}")
print(f"Accuracy:  {perf.accuracy()[0][1]:.4f}")
print(f"Precision: {perf.precision()[0][1]:.4f}")
print(f"Recall:    {perf.recall()[0][1]:.4f}")
print(f"F1 Score:  {perf.F1()[0][1]:.4f}")
print("="*70)

## Confusion Matrix

In [ ]:
# Confusion matrix
cm = perf.confusion_matrix()
print("\n📊 CONFUSION MATRIX")
print("="*70)
print(cm)
print("="*70)

# Extract values
cm_table = cm.table.as_data_frame()
try:
    tn = int(cm_table.iloc[0, 1])
    fp = int(cm_table.iloc[0, 2])
    fn = int(cm_table.iloc[1, 1])
    tp = int(cm_table.iloc[1, 2])

    print("\n💡 BREAKDOWN")
    print(f"True Negatives:  {tn:>6,}  ✅")
    print(f"False Positives: {fp:>6,}  ⚠️")
    print(f"False Negatives: {fn:>6,}  ❌ (Missed frauds)")
    print(f"True Positives:  {tp:>6,}  ✅")
    print(f"\n🎯 Caught {tp} frauds, missed {fn} frauds")
except:
    print("See confusion matrix table above")

## Feature Importance

In [ ]:
# Top features
varimp = best.varimp(use_pandas=True)
print("\n📊 TOP FEATURES")
print("="*70)
print(varimp.head(10))
print("="*70)

# Plot
best.varimp_plot()

## Save Model

In [ ]:
# Save model
model_path = h2o.save_model(model=best, path="./", force=True)
print(f"\n✅ Model saved: {model_path}")

# Download
files.download(model_path)
print("📥 Model downloaded!")

## Quick Summary Report

In [ ]:
# Generate summary
report = f"""
FRAUD DETECTION - QUICK SUMMARY
{'='*60}

Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Dataset:
  Total:     {len(data):,}
  Training:  {train.nrows:,}
  Test:      {test.nrows:,}

Model:
  Best:      {best.model_id}
  Algorithm: {best.algo}
  Training:  {duration/60:.1f} minutes

Performance:
  AUC:       {perf.auc():.4f}
  Accuracy:  {perf.accuracy()[0][1]:.4f}
  Precision: {perf.precision()[0][1]:.4f}
  Recall:    {perf.recall()[0][1]:.4f}
  F1:        {perf.F1()[0][1]:.4f}

Model saved: {model_path}

{'='*60}
"""

print(report)

# Save report
with open('fraud_summary.txt', 'w') as f:
    f.write(report)

files.download('fraud_summary.txt')
print("✅ Summary downloaded!")

## 🎉 Complete!

### What You Got:
- ✅ Trained model (saved & downloaded)
- ✅ Performance metrics
- ✅ Confusion matrix
- ✅ Feature importance
- ✅ Summary report

### Training Time: ~10-15 minutes

### Models Included:
1. **GBM** (Gradient Boosting) - Usually best for fraud
2. **XGBoost** - Fast and accurate
3. **DRF** (Random Forest) - Tree-based ensemble
4. **GLM** (Linear Model) - Fast baseline
5. **DeepLearning** - Neural network

**Note:** Pure Decision Trees aren't included because ensemble methods (GBM, XGBoost, DRF) always perform better. DRF is a forest of decision trees - much more powerful!

### Need to go even faster?
- Reduce to `max_models=3`
- Use only `['GBM', 'XGBoost']`
- Set `max_runtime_secs=600` (10 min)